### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
7169,2023-11-18,Brasil Nbb,16:00,Botafogo,Cerrado,1.65,2.20,153.5,1.87,1.89,-3.5,1.97,1.74
7170,2023-11-18,Brasil Nbb,17:00,Minas,Franca,1.70,2.12,166.5,1.80,1.86,-3.5,2.06,1.68
7171,2023-11-18,Espanha Acb,14:00,Zaragoza,Gran Canaria,1.94,1.90,161.5,1.86,1.94,-1.5,2.05,1.74
7172,2023-11-18,Espanha Acb,16:45,Basquet Girona,Breogan,1.50,2.69,155.5,1.88,1.88,-6.5,2.09,1.69
7173,2023-11-18,Espanha Acb,16:45,Tenerife,Palencia,1.18,5.30,155.5,1.87,1.91,-12.5,2.06,1.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7455,2023-11-18,República Tcheca Zbl Feminina,13:00,USK Prague F,Brandys nad Labem F,1.67,2.03,151.5,1.82,1.84,-57.5,1.90,1.77
7456,2023-11-18,Sérvia 1. Zls Feminino,14:30,Nis F,ZKK Kraljevo F,8.05,1.05,141.5,1.83,1.83,15.5,1.83,1.83
7457,2023-11-18,Sérvia 1. Zls Feminino,16:00,Subotica F,Partizan F,5.37,1.12,146.5,1.83,1.83,11.5,1.88,1.78
7458,2023-11-18,Suécia Basketligan,13:04,Norrkoping,Umeå IK,1.01,13.70,172.5,1.83,1.83,-29.5,1.94,1.77


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
5,00:00,Eua Nba,Portland Trail Blazers,Los Angeles Lakers,224.5,1.80,1.0000,Over
6,00:00,Eua Nba,Utah Jazz,Phoenix Suns,235.5,1.89,1.0000,Over
91,16:00,Eua Ncaa,VCU Rams,Seattle,134.5,2.00,0.8406,Over
114,13:00,Finlândia Korisliiga,UU-Korihait,Kouvot,176.5,1.87,0.6990,Over
255,20:00,Eua Nba,Charlotte Hornets,New York Knicks,221.5,1.80,1.0000,Over
256,21:00,Eua Nba,New Orleans Pelicans,Minnesota Timberwolves,218.5,1.80,0.9998,Over
257,22:00,Eua Nba,Chicago Bulls,Miami Heat,211.5,1.86,1.0000,Over
258,22:00,Eua Nba,San Antonio Spurs,Memphis Grizzlies,224.5,1.81,0.9999,Over
280,19:45,Eua Ncaa,Florida Gulf Coast,Hampton,148.5,1.91,0.8482,Over
